7.
Study the implementation in Recommender Systems in Python 101 | Kaggle  
https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101

where the content based filtering is built using tf-idf vector of the message content https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101?scriptVersionId=24758977&cellId=35
replace the tf-idf by a simple sentiment analyzer result using vader

Suggest a script that implements this paradigm and perform its evaluation using Recall@N and NDCG@N metrics.

In [ ]:
#####
####  Task 7
#### "7.	Study the implementation in Recommender Systems in Python 101 | Kaggle
#### where the content based filtering is built using tf-idf vector of the
#### message content, we want to replace the tf-idf by a simple
#### sentiment analyzer result using vader. Suggest a script that
#### implements this paradigm and perform its evaluation
#### using Recall@N and NDCG@N metrics."
#####

# Initiate necessary tools
# (including vader)
# Connect with data

# determine positive/ neutral/ negative ratio for ARTICLE_TEXT entries and save this new data for later use
##### ( vader can give us evaluations like  {'neg': 0.035, 'neu': 0.877, 'pos': 0.087, 'compound': 0.9611}
##### based on what words are featured prominently in the text. Is it mostly negative, neutral or positive?
##### Details on the scoring: https://github.com/cjhutto/vaderSentiment#about-the-scoring )

# Might want to only process articles that have at least two different users
# interacting with them, otherwise there are no links between some of the data

# FUNC2 (articleID, num_of_results)
# Create function that takes 1 article ID as input and returns N of the most similar articles
# based on smallest distance
# --> similar style of expression in ARTICLE_TEXT entries can be pointed out
#     and we can inspect if people are likely to interact with entries
#     that would be grouped together based on the vader evaluation scores.

# Evaluating the recommender system with Recall@N and NDCG@N

# Recall@N = Number of relevant items in top N recommendations / Total number of relevant items
# We declare an article relevant to a user if they have interacted with the article in any way ( = known to be interested )
# So for each user that has interacted with articles at least N+1 times, we can
# get list of the articles they interacted with,
# and for each item :
#   look up N recommended articles
#   then verify if any of those recommended articles are also ones that the same user has interacted with.
# Number of relevant items in top N recommendations per user's interacted article / Total number of articles interacted by user = Recall@N


# NDCG@N
# ? https://gist.github.com/bwhite/3726239  (has some resource links)




In [ ]:
# initiate necessary tools / import panda etc (same as for earlier tasks can be used)
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats import pearsonr
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
%pip install vaderSentiment #colab requires install
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# Connect with data (same as earlier tasks can be used)
##articles = pd.read_csv("shared_articles.csv", delimiter= ',')
##interactions = pd.read_csv("users_interactions.csv", delimiter= ',')

articles_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp project/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

interactions_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp project/users_interactions.csv')
interactions_df.head(5)

comments_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp project/comments.csv')
comments_df.head(5)



In [180]:
# we determine positive/ neutral/ negative ratio for article text only
# we have original data in articles_df , interactions_df, comments_df
# leave out articles in languages other than english

# (also just take first 100 for quick testing)
articles_en = articles_df[articles_df['lang'] == 'en'].head(100)

vader = SentimentIntensityAnalyzer()
def calculate_compound(text):
    sentiment_scores = vader.polarity_scores(text)
    return sentiment_scores['compound']

# run calculate_compound on each of the dataframe row's "text" column, save it to column "compound"
articles_en['compound'] = articles_en['text'].apply(calculate_compound)

In [188]:
def find_closest(contentid, number_of_rows=5):
  # Find the 'compound' value for the row with correct "contentId" value
  target_compound = articles_en.loc[articles_en['contentId'] == contentid, 'compound'].iloc[0]

  # Calculate the absolute difference between the target 'compound' value and all other rows
  articles_en['compound_diff'] = abs(articles_en['compound'] - target_compound)

  # Sort the DataFrame based on 'compound_diff' and exclude the target row,
  # to get just the 5 closest compound values
  closest_rows = articles_en.sort_values('compound_diff').iloc[1:number_of_rows+1]
  return closest_rows

# find closest rows per contentId
testrows = find_closest(-7292285110016212249, 5)
display(testrows)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,compound,compound_diff
92,1459369124,CONTENT SHARED,8565115980515859979,3891637997717104548,1776955132850411647,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/c...,Calculating and searching 500 billion digits o...,It's Pi day! Have you ever wondered whether yo...,en,0.9573,0.0038
19,1459251714,CONTENT SHARED,7905485530310717815,4340306774493623681,-7292854281461484137,NaN,NaN,NaN,HTML,http://www.coindesk.com/blockchain-smart-contr...,Blockchain Smart Contracts Startup Selected By...,"CommonAccord, a blockchain-based startup for l...",en,0.9712,0.0101
112,1459423524,CONTENT SHARED,-6713697716910928909,4340306774493623681,3903381901308718595,NaN,NaN,NaN,HTML,https://blog.ethereum.org/2016/03/30/solidity-...,Solidity Available in Visual Studio - Ethereum...,Consensys and Microsoft have announced that th...,en,0.9509,0.0102
47,1459302985,CONTENT SHARED,-4994468824009200256,-1443636648652872475,8209530310193218854,NaN,NaN,NaN,HTML,https://www.linkedin.com/pulse/so-your-ai-bot-...,"So your AI bot went haywire, should you care?",A Microsoft chatbot went rogue on Twitter and ...,en,0.9722,0.0111
53,1459340708,CONTENT SHARED,809601605585939618,8414731042150985013,457340264983454091,NaN,NaN,NaN,HTML,http://www.zdnet.com/article/microsoft-and-can...,​Microsoft and Canonical partner to bring Ubun...,"According to sources at Canonical , Ubuntu Lin...",en,0.9725,0.0114


In [ ]:
# Evaluating the recommender system with Recall@N and NDCG@N

# Recall@N = Number of relevant items in top N recommendations / Total number of relevant items
# We declare an article relevant to a user if they have interacted with the article in any way ( = known to be interested )
# So for each user that has interacted with articles at least N+1 times, we can
# get list of the articles they interacted with,
# and for each item :
#   look up N recommended articles
#   then verify if any of those recommended articles are also ones that the same user has interacted with.
# Number of relevant items in top N recommendations per user's interacted article / Total number of articles interacted by user = Recall@N



TASK 8
repeat 7) when the content of message is constituted by a vector corresponding to the concatenation of [Positive Sentiment score, Negative Sentiment Score, Length of the message, Ratio of stopwords /uncommon characters, Number of Personal Pronouns].

In [ ]:
## Task 8
# vector corresponding to the concatenation of
 # Positive Sentiment score (vader)
 # Negative Sentiment Score (vader)
 # Length of the message
 # Ratio of stopwords / uncommon characters
 # Number of Personal Pronouns

switch to using nltk cosine similarity ??